In [0]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.5


In [0]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

# Instala o PyTorch
!pip install -q torch 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b7049fb7-a7ba-44da-b9f3-0b3051e58e52/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b7049fb7-a7ba-44da-b9f3-0b3051e58e52/bin/python -m pip install --upgrade pip' command.


In [0]:
# Imports
import torch
import torch.nn as nn
import numpy as np

In [0]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Eric Passos" --iversions

Author: Eric Passos

numpy: 1.20.3
torch: 1.12.1+cpu



### Preparación de los datos
El siguiente texto es un ejemplo de una declaración de embargo. Si bien el texto representa un embargo, los datos críticos han sido reemplazados por información genérica, lo que no compromete el objetivo del estudio de caso.

In [0]:
# Texto de embargo de declaração
embargo = """O embargante sofreu o ajuizamento de ação de danos morais e materiais, cujo objeto é o reaver os 
valores pagos pelo sinal dado em um contrato de compra e venda de imóvel no qual não foi dado continuidade. Em 24 
de fevereiro de 2012, o Magistrado proferiu decisão de fls. 277 a 280, que condenou todas as demandadas 
solidariamente no seguinte teor: Diante de todo o exposto, com fundamento no art. 1234, I, do CPC/2015, 
julgo procedentes em parte os pedidos constantes na inicial, condenando solidariamente as demandadas, XPTO LTDA, 
BOB CAMARGO DE MORAES, a Pagarema título de indenização por danos morais, consoante fundamentação acima discorrida, 
o montante de R$ 1.500,00 (um mil e quinhentos reais), corrigidos monetariamente pelo INPC desde a data 
desta decisão, acrescidos de juros de 1% ao mês, a partir da citação; condeno ainda, à restituição do valor 
pago pelo demandante como sinal da entrada do imóvel, descontando apenas 20% (vinte por cento), referente às 
despesas, devendo incidir juros de 1% (um por cento) ao mês contados da citação e correção monetária pelo INPC a 
partir da sentença. Contudo, data venia, houve omissão e obscuridade na referida decisão, haja vista que a omissão 
se deu pela ausência dos julgamentos das preliminares (Necessidade de Perícia Técnica e a incompetência de 
Juizado Especial) proposta posteriormente em aditamento de contestação (Fls 251 a 254) para impugnar áudios 
juntados pelo embargado, autorizado a ser realizada pela Douta Magistrada em audiência de Conciliação, 
instrução e julgamento de fls 235 e 236, por ausência de intimação anterior para realizar a já tratada 
impugnação aos áudios anexados."""

In [0]:
# Limpeza do texto substituindo vírgulas e pontos por espaços e colocando as palavras em minúsculo
embargo = embargo.replace(',','').replace('.','').lower().split()

# Criação do corpus com o texto acima
corpus = set(embargo)

# Visualizamos o corpus
corpus

Out[7]: {'(fls',
 '(necessidade',
 '(um',
 '(vinte',
 '1%',
 '1234',
 '150000',
 '20%',
 '2012',
 '235',
 '236',
 '24',
 '251',
 '254)',
 '277',
 '280',
 'a',
 'acima',
 'acrescidos',
 'aditamento',
 'ainda',
 'ajuizamento',
 'anexados',
 'anterior',
 'ao',
 'aos',
 'apenas',
 'art',
 'as',
 'audiência',
 'ausência',
 'autorizado',
 'ação',
 'bob',
 'camargo',
 'cento)',
 'citação',
 'citação;',
 'com',
 'como',
 'compra',
 'conciliação',
 'condenando',
 'condeno',
 'condenou',
 'consoante',
 'constantes',
 'contados',
 'contestação',
 'continuidade',
 'contrato',
 'contudo',
 'correção',
 'corrigidos',
 'cpc/2015',
 'cujo',
 'da',
 'dado',
 'danos',
 'das',
 'data',
 'de',
 'decisão',
 'demandadas',
 'demandante',
 'descontando',
 'desde',
 'despesas',
 'desta',
 'deu',
 'devendo',
 'diante',
 'discorrida',
 'do',
 'dos',
 'douta',
 'e',
 'em',
 'embargado',
 'embargante',
 'entrada',
 'especial)',
 'exposto',
 'fevereiro',
 'fls',
 'foi',
 'fundamentação',
 'fundamento',
 'haja',
 'h

In [0]:
# Comprimento do corpus
corpus_length = len(corpus)

# Dicionários para TF-IDF
dic_palavra = {}
dic_inverso_palavra = {}

In [0]:
# Loop pelo corpus para criar os dicionários
for i, palavra in enumerate(corpus):
    dic_palavra[palavra] = i
    dic_inverso_palavra[i] = palavra

In [0]:
# Lista para receber os dados
dados = []

# Loop pelo texto par extrair sentenças e palavras
for i in range(2, len(embargo) - 2):
    sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]
    target = embargo[i]
    dados.append((sentence, target))

¿Leíste el código anterior y entendiste lo que se hizo? Tenga en cuenta esta línea:

sentencia = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]

Para una palabra en el índice i, tenemos dos palabras antes y dos después. La palabra en el índice i será nuestro objetivo y la oración estará formada por las dos palabras y dos palabras después de la palabra objetivo.

Después de entrenar el modelo, podremos predecir cada palabra en función de las palabras que la rodean.

Aquí un ejemplo:

In [0]:
# Visualiza os dados
print(dados[3])

(['o', 'ajuizamento', 'ação', 'de'], 'de')


Las cuatro palabras de la lista serán los datos de entrada y la palabra fuera de la lista ('de' en este caso) será la variable de salida.

### Construcción del modelo CBoW

In [0]:
# Vamos definir o comprimento de cada embedding
embedding_length = 20

# Classe para o modelo
class CBoW(torch.nn.Module):

    # Método construtor
    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
        
        # Camada de entrada do modelo para criação da embedding
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        # Camadas lineares
        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
        
        # Camadas de ativação
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    # Passo (forward)
    def forward(self, inputs):
        
        # Aqui definimos a ordem das camadas da rede neural
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    # Obtém a word_emdedding
    def get_word_emdedding(self, word):
        word = torch.LongTensor([dic_palavra[word]])
        return self.embeddings(word).view(1,-1)

In [0]:
# Cria o modelo CBoW
modelo = CBoW(corpus_length, embedding_length)

# Função de custo
loss_function = nn.NLLLoss()

# Otimizador do modelo (backpropagation)
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.01)

In [0]:
# Função para criar o vetor de sentenças, necessário para treinar o modelo
def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype = torch.long)

In [0]:
# Aqui está nosso dicionário de palavras
dic_palavra

Out[15]: {'r$': 0,
 'obscuridade': 1,
 'reaver': 2,
 'moraes': 3,
 'xpto': 4,
 'referente': 5,
 'vista': 6,
 '(necessidade': 7,
 'como': 8,
 'incompetência': 9,
 'condenou': 10,
 'valor': 11,
 'magistrada': 12,
 'montante': 13,
 'desde': 14,
 'instrução': 15,
 'fundamento': 16,
 'as': 17,
 'julgamento': 18,
 'inicial': 19,
 'contudo': 20,
 'teor:': 21,
 'para': 22,
 'impugnar': 23,
 'exposto': 24,
 'discorrida': 25,
 'da': 26,
 'parte': 27,
 'posteriormente': 28,
 'a': 29,
 'procedentes': 30,
 'contados': 31,
 'conciliação': 32,
 'venda': 33,
 'solidariamente': 34,
 'não': 35,
 '235': 36,
 '280': 37,
 'impugnação': 38,
 'haja': 39,
 'ausência': 40,
 'inpc': 41,
 'anexados': 42,
 'embargado': 43,
 'acima': 44,
 'deu': 45,
 'pelo': 46,
 '1234': 47,
 'fundamentação': 48,
 'aditamento': 49,
 'é': 50,
 'monetariamente': 51,
 'despesas': 52,
 'danos': 53,
 'ação': 54,
 'pago': 55,
 '20%': 56,
 'juntados': 57,
 'bob': 58,
 '236': 59,
 'os': 60,
 'consoante': 61,
 'apenas': 62,
 'camargo': 63,

In [0]:
# O dicionário de palavras será convertido em um vetor de sentenças. Aqui um exemplo:
print(make_sentence_vector(['pela','ausência','dos','julgamentos'], dic_palavra))

tensor([ 81,  40, 107, 103])


### Entrenamiento modelo

In [0]:
# Loop por 150 passadas (epochs) de treinamento
for epoch in range(150):
    
    # Inicia o erro da época com 0
    epoch_loss = 0
    
    # Loop pelos dados de entrada (sentence) e saída (target)
    for sentence, target in dados:
        
        # Inicializa os gradientes com zero
        modelo.zero_grad()
        
        # Cria o vetor de sentença com os dados de entrada (que devem estar no dicionário de palavras)
        sentence_vector = make_sentence_vector(sentence, dic_palavra)  
        
        # Usa o vetor para fazer previsões com o modelo e retorna as probabilidades
        log_probs = modelo(sentence_vector)
        
        # Calcula o erro do modelo
        loss = loss_function(log_probs, torch.tensor([dic_palavra[target]], dtype = torch.long))
        
        # Chama o método de backpropagation para calcular o gradiente da derivada
        loss.backward()
        
        # Otimiza os pesos do modelo e segue para a próxima passada
        # É aqui que o aprendizado acontece
        optimizer.step()
        
        # Atualiza o erro da época
        epoch_loss += loss.data
        
    # Imprime epoch e erro da epoch    
    print('Epoch: ' + str(epoch) + ', Erro do Modelo: ' + str(epoch_loss.item()))

Epoch: 0, Erro do Modelo: 1347.822265625
Epoch: 1, Erro do Modelo: 1229.6187744140625
Epoch: 2, Erro do Modelo: 1136.5601806640625
Epoch: 3, Erro do Modelo: 1044.5963134765625
Epoch: 4, Erro do Modelo: 947.2789306640625
Epoch: 5, Erro do Modelo: 842.686279296875
Epoch: 6, Erro do Modelo: 732.420166015625
Epoch: 7, Erro do Modelo: 619.4020385742188
Epoch: 8, Erro do Modelo: 508.1355285644531
Epoch: 9, Erro do Modelo: 405.05877685546875
Epoch: 10, Erro do Modelo: 314.5666809082031
Epoch: 11, Erro do Modelo: 239.31234741210938
Epoch: 12, Erro do Modelo: 179.80812072753906
Epoch: 13, Erro do Modelo: 134.8692169189453
Epoch: 14, Erro do Modelo: 101.64507293701172
Epoch: 15, Erro do Modelo: 77.64651489257812
Epoch: 16, Erro do Modelo: 60.70416259765625
Epoch: 17, Erro do Modelo: 48.62139129638672
Epoch: 18, Erro do Modelo: 39.89267349243164
Epoch: 19, Erro do Modelo: 33.39823532104492
Epoch: 20, Erro do Modelo: 28.52992820739746
Epoch: 21, Erro do Modelo: 24.784404754638672
Epoch: 22, Erro d

Nótese cómo se ha ido reduciendo el error con cada pasada, produciéndose claramente un aprendizaje. Ahora usemos el modelo para hacer predicciones.

In [0]:
# Função para obter uma previsão
def get_resultado_previsto(input, dic_inverso_palavra):
    index = np.argmax(input)
    return dic_inverso_palavra[index]

In [0]:
# Função para prever sentenças (aplicamos aos novos dados o mesmo tratamento usado nos dados de treino)
def preve_sentenca(sentence):
    
    # Dividimos a sentença com split
    sentence_split = sentence.replace('.','').lower().split()
    
    # Criamos o vetor de sentença
    sentence_vector = make_sentence_vector(sentence_split, dic_palavra)
    
    # Faz a previsão com o modelo
    prediction_array = modelo(sentence_vector).data.numpy()
    
    # Print dos resultados
    print('Palavras Anteriores: {}\n'.format(sentence_split[:2]))
    print('Palavra Prevista: {}\n'.format(get_resultado_previsto(prediction_array[0], dic_inverso_palavra)))
    print('Palavras Seguintes: {}\n'.format(sentence_split[2:]))

### Predicciones con el modelo

Dentro de la oración: **"ausência de intimação anterior para realizar"**, veamos si el modelo puede predecir la palabra.

Voy a omitir la palabra **intimação** y esa debería ser la palabra predicha por el modelo. Pasemos como entrada las dos palabras anteriores y las dos posteriores.

In [0]:
# Previsão com o modelo
preve_sentenca('ausência de anterior para')

Palavras Anteriores: ['ausência', 'de']

Palavra Prevista: intimação

Palavras Seguintes: ['anterior', 'para']



In [0]:
# Emdedding da palavra
print(modelo.get_word_emdedding('intimação'))

tensor([[-0.4916, -0.9850, -0.6488,  0.0961,  1.3510, -1.1238, -0.3378,  0.8000,
         -0.4194, -0.5021, -0.5944, -1.2084,  0.4076, -0.9108,  0.2849,  0.4788,
          1.3720, -0.0715, -0.0460, -1.6440]], grad_fn=<ViewBackward0>)


¡Perfecto! El modelo predijo la sentencia en la Declaración de Embargo! Un ejemplo más.

Dentro de la oración: **"devendo incidir juros de 1%"**, veamos si el modelo puede predecir la palabra.

Voy a omitir la palabra **juros** y esa debería ser la palabra predicha por el modelo. Pasemos como entrada las dos palabras anteriores y las dos posteriores.

In [0]:
# Previsão com o modelo
preve_sentenca('devendo incidir de 1%')

Palavras Anteriores: ['devendo', 'incidir']

Palavra Prevista: juros

Palavras Seguintes: ['de', '1%']



¡Perfecto! El modelo predijo la sentencia en la Declaración de Embargo! Y CBoW no es el modelo más avanzado en PNL.

# Fin